## Recurrent Neural Network
Predict daily high and low temperatures?

In [83]:
import pandas as pd
import numpy as np
import tensorflow as tf

# workaround for MacOS/jupyter notebook bug w/ tensorflow
# https://www.programmersought.com/article/69923598438/
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [130]:
np.random.seed(4471)
# https://www.tensorflow.org/tutorials/load_data/numpy
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
weather_pd = pd.read_csv('../data/weather.csv', index_col = 0)
weather_pd = weather_pd.drop(['DAY', 'STP', 'GUST'], axis=1)
weather_np = weather_pd.to_numpy()

In [85]:
weather_pd.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0


In [86]:
weather_pd.dtypes

YEAR        int64
MONTH       int64
SEASON      int64
TEMP      float64
DEWP      float64
SLP       float64
VISIB     float64
WDSP      float64
MXSPD     float64
MAX       float64
MIN       float64
PRCP      float64
SNDP      float64
dtype: object

In [87]:
# next day's weather
target = weather_pd[['MAX']].iloc[1:, :] # 'MIN'
target = np.round(target.values.reshape(-1))
print(target.shape)
target

(7771,)


array([70., 74., 74., ..., 87., 85., 58.])

In [88]:
# today's weather
feature = weather_pd.iloc[:-1, :] # don't have next day on last day
feature.head()

,YEAR,MONTH,SEASON,TEMP,DEWP,SLP,VISIB,WDSP,MXSPD,MAX,MIN,PRCP,SNDP
0,2000,1,0,47.6,38.1,1023.7,8.3,3.0,10.1,66.9,33.1,0.00,0.0
1,2000,1,0,55.3,46.3,1024.2,9.5,4.8,14.0,70.0,33.1,0.00,0.0
2,2000,1,0,62.6,55.4,1021.3,8.4,8.5,14.0,73.9,43.0,0.00,0.0
3,2000,1,0,65.2,58.6,1014.4,9.5,15.3,28.0,73.9,55.0,0.00,0.0
4,2000,1,0,45.7,30.9,1019.8,9.8,6.4,11.1,57.9,37.0,0.34,0.0


In [89]:
weather = tf.data.Dataset.from_tensor_slices((feature.values, target))

In [90]:
for feat, targ in weather.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [2.0000e+03 1.0000e+00 0.0000e+00 4.7600e+01 3.8100e+01 1.0237e+03
 8.3000e+00 3.0000e+00 1.0100e+01 6.6900e+01 3.3100e+01 0.0000e+00
 0.0000e+00], Target: 70.0
Features: [2.0000e+03 1.0000e+00 0.0000e+00 5.5300e+01 4.6300e+01 1.0242e+03
 9.5000e+00 4.8000e+00 1.4000e+01 7.0000e+01 3.3100e+01 0.0000e+00
 0.0000e+00], Target: 74.0
Features: [2.0000e+03 1.0000e+00 0.0000e+00 6.2600e+01 5.5400e+01 1.0213e+03
 8.4000e+00 8.5000e+00 1.4000e+01 7.3900e+01 4.3000e+01 0.0000e+00
 0.0000e+00], Target: 74.0
Features: [2.0000e+03 1.0000e+00 0.0000e+00 6.5200e+01 5.8600e+01 1.0144e+03
 9.5000e+00 1.5300e+01 2.8000e+01 7.3900e+01 5.5000e+01 0.0000e+00
 0.0000e+00], Target: 58.0
Features: [2.0000e+03 1.0000e+00 0.0000e+00 4.5700e+01 3.0900e+01 1.0198e+03
 9.8000e+00 6.4000e+00 1.1100e+01 5.7900e+01 3.7000e+01 3.4000e-01
 0.0000e+00], Target: 51.0


In [91]:
# batches of days - ues previous 3 days to predict 4th day?
days_in_batches = 4 # change to 3 later
batches = weather.batch(days_in_batches, drop_remainder=True)
batches

<BatchDataset shapes: ((4, 13), (4,)), types: (tf.float64, tf.float64)>

In [92]:
# see batches of days
for input_ex, target_ex in batches.take(5):
    print('Input data: ', input_ex)
    print('Target data: ', target_ex)

Input data:  tf.Tensor(
[[2.0000e+03 1.0000e+00 0.0000e+00 4.7600e+01 3.8100e+01 1.0237e+03
  8.3000e+00 3.0000e+00 1.0100e+01 6.6900e+01 3.3100e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 5.5300e+01 4.6300e+01 1.0242e+03
  9.5000e+00 4.8000e+00 1.4000e+01 7.0000e+01 3.3100e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 6.2600e+01 5.5400e+01 1.0213e+03
  8.4000e+00 8.5000e+00 1.4000e+01 7.3900e+01 4.3000e+01 0.0000e+00
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 6.5200e+01 5.8600e+01 1.0144e+03
  9.5000e+00 1.5300e+01 2.8000e+01 7.3900e+01 5.5000e+01 0.0000e+00
  0.0000e+00]], shape=(4, 13), dtype=float64)
Target data:  tf.Tensor([70. 74. 74. 58.], shape=(4,), dtype=float64)
Input data:  tf.Tensor(
[[2.0000e+03 1.0000e+00 0.0000e+00 4.5700e+01 3.0900e+01 1.0198e+03
  9.8000e+00 6.4000e+00 1.1100e+01 5.7900e+01 3.7000e+01 3.4000e-01
  0.0000e+00]
 [2.0000e+03 1.0000e+00 0.0000e+00 3.6100e+01 2.7200e+01 1.0324e+03
  9.9000e+00 1.8000e+00 7.0000e+

In [93]:
# see expected output for each input
for i, (input_idx, target_idx) in enumerate(zip(input_ex[:5], target_ex[:5])):
    print("Step {:4d}".format(i))
    print("  input: {}".format(input_idx))
    print("  expected output: {}".format(target_idx))

Step    0
  input: [2.000e+03 1.000e+00 0.000e+00 4.300e+01 1.550e+01 1.027e+03 9.900e+00
 6.800e+00 1.500e+01 6.300e+01 3.200e+01 0.000e+00 0.000e+00]
  expected output: 33.0
Step    1
  input: [2.0000e+03 1.0000e+00 0.0000e+00 2.6600e+01 1.4000e+01 1.0235e+03
 5.1000e+00 3.6000e+00 8.0000e+00 3.3100e+01 2.1200e+01 9.0000e-02
 3.1000e+00]
  expected output: 40.0
Step    2
  input: [2.0000e+03 1.0000e+00 0.0000e+00 3.0500e+01 2.7200e+01 1.0168e+03
 5.4000e+00 2.7000e+00 8.0000e+00 3.9900e+01 2.1900e+01 1.0000e-01
 2.0000e+00]
  expected output: 45.0
Step    3
  input: [2.0000e+03 1.0000e+00 0.0000e+00 3.5700e+01 2.8400e+01 1.0061e+03
 6.3000e+00 7.1000e+00 1.5000e+01 4.5000e+01 2.4100e+01 2.6000e-01
 2.0000e+00]
  expected output: 45.0


In [94]:
# make batches of the 4-day batches
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

batch_weather = batches.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

batch_weather

<BatchDataset shapes: ((64, 4, 13), (64, 4)), types: (tf.float64, tf.float64)>

## Build the Model

In [111]:
# how many possible outputs
num_output_vals = int(max(target)) + 5 # maximum temperature observed (105) plus some margin

# Number of RNN units
rnn_units = 16

In [112]:
def build_model(rnn_units, num_output_vals, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform',
                            batch_input_shape = (64, 4, 13)),
        tf.keras.layers.Dense(num_output_vals)
    ])
    return model

In [113]:
model = build_model(rnn_units, num_output_vals, BATCH_SIZE)

In [133]:
# set floats to float64 to avoid warning message
tf.keras.backend.set_floatx('float64')

## Try the Model

In [114]:
for input_example_batch, target_example_batch in batch_weather.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, days_in_batches, num_output_vals)")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(64, 4, 110) # (batch_size, days_in_batches, num_output_vals)


In [115]:
target_example_batch.shape # 512 total # need another dim??

TensorShape([64, 4])

In [116]:
input_example_batch.shape # 64*52=3328

TensorShape([64, 4, 13])

In [117]:
example_batch_predictions.shape

TensorShape([64, 4, 110])

In [118]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (64, 4, 16)               1488      
_________________________________________________________________
dense_5 (Dense)              (64, 4, 110)              1870      
Total params: 3,358
Trainable params: 3,358
Non-trainable params: 0
_________________________________________________________________


In [119]:
# example predictions, all terrible bc model not trained
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 5, 84,  1, 71])

In [120]:
# show input and output
print("Input: \n", input_example_batch[0])
print()
print("Next Day Predictions: \n", sampled_indices)

Input: 
 tf.Tensor(
[[2.0050e+03 2.0000e+00 0.0000e+00 3.6900e+01 2.8300e+01 1.0260e+03
  6.6000e+00 5.6000e+00 8.9000e+00 4.8000e+01 2.5000e+01 3.0000e-02
  0.0000e+00]
 [2.0050e+03 2.0000e+00 0.0000e+00 3.7800e+01 3.1800e+01 1.0230e+03
  5.2000e+00 6.4000e+00 1.3000e+01 4.8900e+01 3.2000e+01 5.0000e-01
  0.0000e+00]
 [2.0050e+03 2.0000e+00 0.0000e+00 4.4200e+01 2.8400e+01 1.0264e+03
  1.0000e+01 5.3000e+00 1.2000e+01 5.9000e+01 3.2000e+01 0.0000e+00
  0.0000e+00]
 [2.0050e+03 2.0000e+00 0.0000e+00 4.3800e+01 2.7300e+01 1.0311e+03
  1.0000e+01 4.1000e+00 1.1100e+01 6.2100e+01 2.8000e+01 0.0000e+00
  0.0000e+00]], shape=(4, 13), dtype=float64)

Next Day Predictions: 
 [ 5 84  1 71]


In [121]:
# do we need logits for loss function? from_logits=True
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, days_in_batches, num_output_vals)")
print("Mean loss:        ", mean_loss)
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

Prediction shape:  (64, 4, 110)  # (batch_size, days_in_batches, num_output_vals)
Mean loss:         4.6801605


In [122]:
# exponential of the mean loss ~= num outputs
tf.exp(mean_loss).numpy()

107.78738

In [123]:
model.compile(optimizer='adam', loss=loss)

## Configure Checkpoints
save checkpoints during training

In [124]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training-checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Train Model

In [125]:
EPOCHS = 30

In [126]:
history = model.fit(batch_weather, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
30/30 [==============================] - 3s 84ms/step - loss: 4.5788
Epoch 2/30
30/30 [==============================] - 0s 13ms/step - loss: 4.4154
Epoch 3/30
30/30 [==============================] - 0s 13ms/step - loss: 4.3084
Epoch 4/30
30/30 [==============================] - 0s 13ms/step - loss: 4.2402
Epoch 5/30
30/30 [==============================] - 0s 13ms/step - loss: 4.1995
Epoch 6/30
30/30 [==============================] - 0s 13ms/step - loss: 4.1690
Epoch 7/30
30/30 [==============================] - 0s 14ms/step - loss: 4.1505
Epoch 8/30
30/30 [==============================] - 0s 14ms/step - loss: 4.1349
Epoch 9/30
30/30 [==============================] - 0s 13ms/step - loss: 4.1223
Epoch 10/30
30/30 [==============================] - 0s 14ms/step - loss: 4.1113
Epoch 11/30
30/30 [==============================] - 0s 13ms/step - loss: 4.1103
Epoch 12/30
30/30 [==============================] - 0s 14ms/step - loss: 4.1056
Epoch 13/30
30/30 [==================

## Restore Latest Checkpoint

In [127]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(rnn_units, num_output_vals, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (64, 4, 16)               1488      
_________________________________________________________________
dense_6 (Dense)              (64, 4, 110)              1870      
Total params: 3,358
Trainable params: 3,358
Non-trainable params: 0
_________________________________________________________________


In [140]:
# weather_np[0, :]
input_example_batch[0]

<tf.Tensor: id=17464, shape=(4, 13), dtype=float64, numpy=
array([[2.0050e+03, 2.0000e+00, 0.0000e+00, 3.6900e+01, 2.8300e+01,
        1.0260e+03, 6.6000e+00, 5.6000e+00, 8.9000e+00, 4.8000e+01,
        2.5000e+01, 3.0000e-02, 0.0000e+00],
       [2.0050e+03, 2.0000e+00, 0.0000e+00, 3.7800e+01, 3.1800e+01,
        1.0230e+03, 5.2000e+00, 6.4000e+00, 1.3000e+01, 4.8900e+01,
        3.2000e+01, 5.0000e-01, 0.0000e+00],
       [2.0050e+03, 2.0000e+00, 0.0000e+00, 4.4200e+01, 2.8400e+01,
        1.0264e+03, 1.0000e+01, 5.3000e+00, 1.2000e+01, 5.9000e+01,
        3.2000e+01, 0.0000e+00, 0.0000e+00],
       [2.0050e+03, 2.0000e+00, 0.0000e+00, 4.3800e+01, 2.7300e+01,
        1.0311e+03, 1.0000e+01, 4.1000e+00, 1.1100e+01, 6.2100e+01,
        2.8000e+01, 0.0000e+00, 0.0000e+00]])>

In [142]:
model.reset_states()
model(input_example_batch[0])

ValueError: Shape (13, 4) must have rank at least 3

In [128]:
def generate_weather(model, start_weather):
    # Evaluation step (generating weather using the learned model)

    # Number of days to generate
    num_generate = 20

    # Converting our start string to numbers (vectorizing)
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    days_generated = []

    # Low temperatures results in more predictable results.
    # Higher temperatures results in more surprising results.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(start_weather)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the temperature returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted temp as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        days_generated.append(idx2char[predicted_id])

    return (start_weather, text_generated)

In [ ]:
print(generate_weather(model, start_weather=))

#### Resources Consulted
https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf